# Part 2: Semantic Kernel Fundamentals

This notebook demonstrates Semantic Kernel's core capabilities using Azure OpenAI. We'll explore the plugin architecture, kernel orchestration, and structured function calling that showcase SK's "Orchestrate Everything" philosophy.

## Environment Setup

First, let's load our environment variables and configure our Azure OpenAI connection for Semantic Kernel.

In [ ]:
# Environment and configuration setup
import os
import warnings
import asyncio
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

class SemanticKernelConfig:
    """Configuration management for Azure OpenAI in Semantic Kernel"""
    
    def __init__(self):
        # Load environment variables
        env_path = find_dotenv()
        if env_path:
            load_dotenv(env_path)
            print(f"✅ Loaded environment from: {env_path}")
        else:
            warnings.warn("No .env file found. Using system environment variables only.")
        
        self._load_azure_config()
        self._validate_config()
    
    def _load_azure_config(self):
        """Load Azure OpenAI configuration from environment variables"""
        self.azure_api_key = os.getenv('AZURE_OPENAI_API_KEY')
        self.azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
        self.azure_deployment = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME', 'gpt-4o-mini')
        self.azure_api_version = os.getenv('AZURE_OPENAI_API_VERSION', '2024-10-21')
        self.service_id = "azure_chat"
    
    def _validate_config(self):
        """Validate critical configuration"""
        errors = []
        
        if not self.azure_api_key:
            errors.append("AZURE_OPENAI_API_KEY is required")
        
        if not self.azure_endpoint:
            errors.append("AZURE_OPENAI_ENDPOINT is required")
        
        if errors:
            raise ValueError(f"Configuration errors: {', '.join(errors)}")
        
        print("✅ All Azure OpenAI configuration validated successfully")
    
    def display_config(self):
        """Display current configuration (hiding secrets)"""
        print("Azure OpenAI Configuration for Semantic Kernel:")
        print(f"  Endpoint: {self.azure_endpoint}")
        print(f"  Deployment: {self.azure_deployment}")
        print(f"  API Version: {self.azure_api_version}")
        print(f"  Service ID: {self.service_id}")
        print(f"  API Key: {'*' * 20 + self.azure_api_key[-4:] if self.azure_api_key else 'Not set'}")

# Initialize configuration
try:
    sk_config = SemanticKernelConfig()
    sk_config.display_config()
except Exception as e:
    print(f"❌ Failed to load configuration: {e}")
    print("\nPlease ensure you have a .env file with the following variables:")
    print("- AZURE_OPENAI_API_KEY")
    print("- AZURE_OPENAI_ENDPOINT")
    print("- AZURE_OPENAI_CHAT_DEPLOYMENT_NAME (optional, defaults to gpt-4o-mini)")
    raise

### 2.1 Basic Chat and Kernel Configuration

Semantic Kernel uses a dependency injection pattern with services registered in the kernel. This provides strong typing and better enterprise patterns.

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.contents import ChatHistory

# Initialize the kernel - central orchestrator for all SK operations
kernel = sk.Kernel()

# Configure Azure OpenAI chat completion service with dependency injection
azure_chat_service = AzureChatCompletion(
    service_id=sk_config.service_id,
    deployment_name=sk_config.azure_deployment,
    endpoint=sk_config.azure_endpoint,
    api_key=sk_config.azure_api_key,
    api_version=sk_config.azure_api_version
)

# Register the service with the kernel (dependency injection pattern)
kernel.add_service(azure_chat_service)

print("🔧 Kernel Configuration:")
print(f"  Services registered: {len(kernel.services)}")
print(f"  Chat service ID: {sk_config.service_id}")

async def demonstrate_basic_chat():
    """Demonstrates basic chat functionality with proper settings"""
    
    # Create chat history for conversation context
    history = ChatHistory()
    history.add_system_message("You are a helpful assistant who provides concise, step-by-step answers.")
    history.add_user_message("What is the capital of France?")
    
    # Create execution settings - required for all SK operations
    execution_settings = AzureChatPromptExecutionSettings(
        service_id=sk_config.service_id,
        max_tokens=150,
        temperature=0.7
    )
    
    print("💬 Basic Chat Example:")
    print("User > What is the capital of France?")
    
    # Execute the chat completion
    response = await azure_chat_service.get_chat_message_content(
        chat_history=history, 
        settings=execution_settings
    )
    
    print(f"Assistant > {response.content}")
    
    # Demonstrate conversation continuation
    history.add_assistant_message(response.content)
    history.add_user_message("What language do they speak there?")
    
    print("\\nUser > What language do they speak there?")
    
    response2 = await azure_chat_service.get_chat_message_content(
        chat_history=history,
        settings=execution_settings
    )
    
    print(f"Assistant > {response2.content}")

# Execute the demonstration
await demonstrate_basic_chat()

### 2.2 Plugin Architecture and Sequential Orchestration

Semantic Kernel's plugin system provides a structured approach to building reusable components. Unlike LangChain's composition, SK uses explicit orchestration.

In [ ]:
# Create semantic functions from prompts (SK's equivalent to LangChain chains)
# These are registered as plugins in the kernel

# Plugin 1: Company Naming
company_naming_prompt = """
What is a good name for a company that makes {{$input}}? 
Provide just the company name, nothing else.
"""

naming_function = kernel.create_function_from_prompt(
    function_name="generate_name", 
    plugin_name="MarketingPlugin", 
    prompt=company_naming_prompt,
    description="Generates creative company names for products"
)

# Plugin 2: Catchphrase Generation  
catchphrase_prompt = """
Write a creative, memorable catchphrase for the following company: {{$input}}
Make it catchy and professional. Provide just the catchphrase, nothing else.
"""

catchphrase_function = kernel.create_function_from_prompt(
    function_name="generate_catchphrase", 
    plugin_name="MarketingPlugin", 
    prompt=catchphrase_prompt,
    description="Creates marketing catchphrases for companies"
)

print("🔌 Plugins Registered:")
print(f"  Total plugins: {len(kernel.plugins)}")
for plugin_name, plugin in kernel.plugins.items():
    print(f"  - {plugin_name}: {len(plugin.functions)} functions")
    for func_name in plugin.functions:
        print(f"    • {func_name}")

async def demonstrate_orchestration():
    """Demonstrates explicit orchestration - SK's approach to sequential processing"""
    
    product = "colorful, eco-friendly socks"
    print(f"\\n🎯 Input product: {product}")
    print("\\n🔗 Running Sequential Orchestration...")
    
    # Step 1: Generate company name (explicit orchestration)
    print("\\n📝 Step 1: Generating company name...")
    try:
        name_result = await kernel.invoke(naming_function, sk.KernelArguments(input=product))
        company_name = str(name_result).strip()
        print(f"   Result: {company_name}")
    except Exception as e:
        print(f"   Error: {e}")
        return
    
    # Step 2: Generate catchphrase using the generated name
    print("\\n🎨 Step 2: Generating catchphrase...")
    try:
        catchphrase_result = await kernel.invoke(catchphrase_function, sk.KernelArguments(input=company_name))
        catchphrase = str(catchphrase_result).strip()
        print(f"   Result: {catchphrase}")
    except Exception as e:
        print(f"   Error: {e}")
        return
    
    # Display final results
    print("\\n" + "="*50)
    print("✅ ORCHESTRATION COMPLETE")
    print("="*50)
    print(f"Product: {product}")
    print(f"Company Name: {company_name}")
    print(f"Catchphrase: {catchphrase}")
    print("="*50)
    
    return {
        "product": product,
        "company_name": company_name, 
        "catchphrase": catchphrase
    }

# Execute the orchestration
result = await demonstrate_orchestration()

# Demonstrate plugin reusability
print("\\n🔄 Demonstrating Plugin Reusability...")
test_products = ["smart home devices", "artisanal coffee beans"]

for product in test_products:
    print(f"\\n→ Testing with: {product}")
    name_result = await kernel.invoke(naming_function, sk.KernelArguments(input=product))
    print(f"  Generated name: {name_result}")

### 2.3 Native Function Plugins

SK's strength lies in its ability to combine AI with traditional code through native function plugins. This demonstrates the hybrid AI + code approach.

In [ ]:
### 2.4 Automatic Function Calling (Basic Agent Behavior)

Semantic Kernel's automatic function calling provides agent-like behavior through structured function invocation. This demonstrates single-agent capabilities before exploring the Agent Framework.

## Summary: Semantic Kernel Fundamentals

This notebook covered Semantic Kernel's core concepts using enterprise-ready patterns:

### Key Concepts Learned:
1. **Kernel Architecture**: Central orchestrator with dependency injection and service registration
2. **Plugin System**: Structured approach combining semantic functions (AI) and native functions (code)
3. **Explicit Orchestration**: Programmatic control flow rather than compositional chaining
4. **Function Calling**: Automatic function selection with typed interfaces and safety

### Semantic Kernel's Philosophy:
- **"Orchestrate Everything"**: Structured orchestration with clear separation of concerns
- **Hybrid AI + Code**: Seamless integration of LLM capabilities with traditional programming
- **Enterprise Patterns**: Dependency injection, typed interfaces, and governance-ready architecture
- **Plugin Ecosystem**: Reusable, discoverable functions with proper metadata

### SK vs LangChain Comparison:
| Aspect | LangChain (LCEL) | Semantic Kernel |
|--------|------------------|-----------------|
| **Composition** | Pipe operators (`\|`) | Explicit `kernel.invoke()` calls |
| **Functions** | Tools with dynamic binding | Plugins with typed interfaces |
| **Flow Control** | Implicit through chains | Explicit orchestration code |
| **Enterprise Focus** | Community-driven | Microsoft enterprise patterns |

### Next Steps:
- **Multi-Agent Systems**: See `4_semantic_kernel_agents.ipynb` for Agent Framework
- **Advanced Plugins**: Memory integration, planning, and complex workflows
- **Production Deployment**: Azure integration, monitoring, and governance

### Production Considerations:
- Leverage dependency injection for testability and maintainability
- Use typed interfaces for better IDE support and error prevention
- Implement proper error handling and logging throughout the kernel
- Consider plugin security and validation for production deployments
- Take advantage of Azure ecosystem integration for enterprise features

In [ ]:
# This cell is being removed - duplicate content

# This cell is being removed - redundant summary